In [2]:
from flask import Flask, render_template, request
import pandas as pd 
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import xgboost 


# app = Flask(__name__)



def cleantext(df): 
    words_remove = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what", "there","all","we",
                    "one","the","a","an","of","or","in","for","by","on","but","is","in","a","not","with","as",
                    "was","if","they","are","this","and","it","have","has","from","at","my","be","by","not","that","to",
                    "from","com","org","like","likes","so","said","from","what","told","over","more","other",
                    "have","last","with","this","that","such","when","been","says","will","also","where","why",
                    "would","today", "in", "on", "you", "r", "d", "u", "hw","wat", "oly", "s", "b", "ht", 
                    "rt", "p","the","th", "n", "was"]

    df['reviews_text'] = df['reviews_text'].replace(r'<ed>','', regex = True)
    df['reviews_text'] = df['reviews_text'].replace(r'\B<U+.*>|<U+.*>\B|<U+.*>','', regex = True)
    
    # convert tweets to lowercase
    df['reviews_text'] = df['reviews_text'].str.lower()
    
    #remove user mentions
    df['reviews_text'] = df['reviews_text'].replace(r'^(@\w+)',"", regex=True)
    
    #remove_symbols
    df['reviews_text'] = df['reviews_text'].replace(r'[^a-zA-Z0-9]', " ", regex=True)

    #remove punctuations 
    df['reviews_text'] = df['reviews_text'].replace(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+',"", regex = True)
    
    #remove words of length 1 or 2 
    df['reviews_text'] = df['reviews_text'].replace(r'\b[a-zA-Z]{1,2}\b','', regex=True)

    #remove extra spaces in the tweet
    df['reviews_text'] = df['reviews_text'].replace(r'^\s+|\s+$'," ", regex=True)
    stop_words = set(stopwords.words('english'))
    mystopwords = [stop_words, "via", words_remove]
    
    df['reviews_text'] = df['reviews_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in mystopwords]))

    df = df.replace({"Positive":1, "Negative":0})
    
    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{1,}',
        stop_words='english',
        ngram_range=(1, 3), 
        max_features=50000
        )
    all_text = df['reviews_text']
    word_vectorizer.fit(all_text)
    train_word_features = word_vectorizer.transform(all_text)

    model = joblib.load("joblib_model_xgb.pkl")
    preds = model.predict(train_word_features)
    df['preds'] = preds
    
    return df




In [3]:
df = pd.read_csv("sample30.csv")
fin_ratings = pd.read_csv("./User_final_ratings.csv")
fin_ratings = fin_ratings.set_index('user_id')

In [4]:
fin_ratings


,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,5302050 15/16 FCT/HOSE ADAPTOR,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,Alex Cross (dvdvideo),...,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee","WeatherTech 40647 14-15 Outlander Cargo Liners Behind 2nd Row, Black",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Wilton Black Dots Standard Baking Cups,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
user_id,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02dakota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02deuce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zulaa118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zuttle,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zwithanx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
 lis = fin_ratings.loc['rony'].sort_values(ascending=False).index[:20]
# df_recom = df[df['name'].isin(lis)]
# fin_df = cleantext(df_recom)
# fin_df = fin_df[['name', 'preds']]
# d = fin_df.groupby('name').mean().sort_values(ascending=False, by="preds")*100

In [8]:
df_recom = df[df['name'].isin(lis)]

In [10]:
fin_df = cleantext(df_recom)

C:\Users\pl54381\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pl54381\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pl54381\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [11]:
fin_df = fin_df[['name', 'preds']]
d = fin_df.groupby('name').mean().sort_values(ascending=False, by="preds")*100